In [45]:
!pip install torchviz
!pip intasll pytorch-model-summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "intasll" - maybe you meant "install"


In [44]:
import os 
import cv2 
import sys 
import sklearn 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import torch.nn as nn
import torch.nn  
import torch.nn.functional as F 


import torchvision 
import torchvision.transforms as transforms 

from sklearn.model_selection import train_test_split 
from torchviz import make_dot
from torchinfo import  

from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader   

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
train_dataset = torchvision.datasets.FashionMNIST(
    "FashionMNIST/", 
    download=True, 
    transform = transforms.Compose([transforms.ToTensor()]))

test_dataset = torchvision.datasets.FashionMNIST(
    "FashionMNIST/", 
    download=True, 
    train=False, 
    transform = transforms.Compose([transforms.ToTensor()]))


In [12]:
train_loader = DataLoader(train_dataset, batch_size = 100)
test_loader = DataLoader(test_dataset, batch_size = 100)


In [14]:
labels_map = {0 : 'T-Shirt', 
              1 : 'Trouser', 
              2 : 'Pullover', 
              3 : 'Dress', 
              4 : 'Coat', 
              5 : 'Sandal', 
              6 : 'Shirt',
              7 : 'Sneaker', 
              8 : 'Bag', 
              9 : 'Ankle Boot'}

In [25]:
def show_version():
    print("=========version=========")
    print(f"python : {sys.version}")
    print(f"torch : {torch.__version__}")
    print(f"numpy : {np.__version__}")
    print(f"pandas  : {pd.__version__}")    
    print(f"sklearn :  {sklearn.__version__}")

def show_grid_images():
    nrows=5 
    ncols=4
    fig = plt.figure(figsize=(12, 12))
    
    for i in range(1, 5*4 +1):
        image_xy = np.random.randint(len(train_dataset))
        image = train_dataset[image_xy][0][0, :, :]
        fig.add_subplot(nrows, ncols, i)
        plt.title(labels_map[train_dataset[image_xy][1]])
        plt.axis("off")
        plt.imshow(image, cmap="gray")
    plt.show()

In [ ]:
show_version()
show_grid_images()

In [51]:
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        # layer1 
        # kernel_size 3 * 3 
        # channel = depth 

        self.layer1 = nn.Sequential( 
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1), 
            nn.BatchNorm2d(32), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(in_features=64 * 6 * 6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) 
        out = self.fc1(out) 
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out 


In [53]:
learning_rate = 0.001
model = FashionCNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [56]:
num_epochs = 5 
count = 0 
# 1 
loss_list = [] 
iteration_list = [] 
accuracy_list = [] 

prediction_list = [] 
labels_list = [] 

for epoch in range(num_epochs):
    # 2 
    for images, labels in train_loader: 
        # 3 
        images , labels  = images.to(device), labels.to(device)

        
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)

        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1 

        if not (count % 50): 
            total = 0 
            correct = 0 

            for images, labels in test_loader: 
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
                test = Variable(images.view(100, 1, 28, 28))
                outputs = model(test)
                predictions = torch.max(outputs, 1)[1].to(device)
                prediction_list.append(predictions)

                correct += (predictions == labels).sum()
                total += len(labels)
            
            # 5 
            accuracy = correct * 100 / total 
            # 1 
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)

        
        if not(count % 500):
            print(f"Iteration : {count} \nLoss : {loss.data} \nAccuracy : {accuracy}")




/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Iteration : 500 
Loss : 0.4936823546886444 
Accuracy : 87.9800033569336
Iteration : 1000 
Loss : 0.3157854676246643 
Accuracy : 89.54000091552734
Iteration : 1500 
Loss : 0.3250182271003723 
Accuracy : 88.54000091552734
Iteration : 2000 
Loss : 0.22408969700336456 
Accuracy : 89.62999725341797
Iteration : 2500 
Loss : 0.144230455160141 
Accuracy : 89.83000183105469
Iteration : 3000 
Loss : 0.18140695989131927 
Accuracy : 90.05000305175781
